In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import rasterio
import numpy as np
import torch.nn as nn
import torch.optim as optim
from skimage.transform import resize
from tqdm import tqdm
import pandas as pd
import torchvision.transforms as transforms

In [5]:
def train_model(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, targets in tqdm(dataloader):
        inputs  = inputs.float().to(device)
        targets = targets.float().to(device)

        inputs[torch.isnan(inputs)] = 0
        targets[torch.isnan(targets)] = 0

        assert not torch.isnan(inputs).any(), "Input contains NaN values"
        assert not torch.isnan(targets).any(), "Target contains NaN values"
        
        optimizer.zero_grad()
        outputs_up = model(inputs)

        loss = criterion(outputs_up, targets).to(device)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

def validate_model(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for inputs, targets in tqdm(dataloader):
            inputs = inputs.float().to(device)
            targets =  targets.float().to(device)
            inputs[torch.isnan(inputs)] = 0
            targets[torch.isnan(targets)] = 0

            outputs = model(inputs)

            loss = criterion(outputs, targets)
            running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss


In [6]:
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, num_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(num_channels, num_channels, kernel_size=3, stride=1, padding=1, bias=True)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(num_channels, num_channels, kernel_size=3, stride=1, padding=1, bias=True)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.relu(out)
        out = self.conv2(out)
        return out + residual  # 残差连接

class EDSR(nn.Module):
    def __init__(self, num_channels=1, num_residuals=16, scale_factor=10):
        super(EDSR, self).__init__()
        
        # 卷积层初始化
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=3, stride=1, padding=1)
        
        # 一系列残差块
        self.residual_blocks = nn.Sequential(
            *[ResidualBlock(64) for _ in range(num_residuals)]
        )
        
        # 卷积层结束
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        
        # 放大块（上采样）
        self.upsample = nn.Sequential(
            nn.Conv2d(64, 64 * (scale_factor ** 2), kernel_size=3, stride=1, padding=1),
            nn.PixelShuffle(scale_factor),  # 像素混洗（PixelShuffle）实现上采样
            nn.Conv2d(64, num_channels, kernel_size=3, stride=1, padding=1)
        )

    def forward(self, x):
        # 初始卷积
        out = self.conv1(x)
        residual = out  # 保存残差
        
        # 残差块部分
        out = self.residual_blocks(out)
        out = self.conv2(out)
        
        # 添加初始残差
        out = out + residual
        
        # 上采样
        out = self.upsample(out)
        return out


In [9]:
model = EDSR(num_channels=1, num_residuals=16, scale_factor=10).to(device)

learning_rate = 0.0001
criterion = nn.L1Loss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-4)

num_epochs = 100
best_loss = float('inf')


for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print(f'Model is training on: {next(model.parameters()).device}')
    train_loss = train_model(model, dataloader_train, criterion, optimizer, device)
    val_loss = validate_model(model, dataloader_val, criterion, device)
    print(f'Train Loss: {train_loss:.4f} | Validation Loss: {val_loss:.4f}')
    
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), 'path')
        print('Model saved!')


Epoch 1/100
Model is training on: cuda:0


100%|██████████| 115/115 [00:30<00:00,  3.83it/s]


Train Loss: 12.0724 | Validation Loss: 4.8563
Model saved!
Epoch 2/100
Model is training on: cuda:0


100%|██████████| 115/115 [00:28<00:00,  3.98it/s]


Train Loss: 3.8449 | Validation Loss: 3.1300
Model saved!
Epoch 3/100
Model is training on: cuda:0


...

Train Loss: 0.9268 | Validation Loss: 0.9205
Epoch 99/100
Model is training on: cuda:0


100%|██████████| 115/115 [00:29<00:00,  3.89it/s]


Train Loss: 0.8789 | Validation Loss: 0.7177
Epoch 100/100
Model is training on: cuda:0


100%|██████████| 115/115 [00:29<00:00,  3.90it/s]

Train Loss: 0.5623 | Validation Loss: 0.5142
Model saved!
